### Preparing data

In [39]:
import os
import random
import shutil
import pandas as pd
import cv2
import numpy as np
import keras

In [2]:
class DataGenerator:
	DATASET_DIR = "C:\\Users\\ASUS\\Documents\\Proyectos\\pokedex\\pokemon"

	def __init__(self):
		self.category_dirs = os.listdir(DataGenerator.DATASET_DIR)
		self.data = pd.DataFrame({}, columns = ['label', 'image']) 
		self.data_train = pd.DataFrame({}, columns = ['label', 'image']) 
		self.data_test = pd.DataFrame({}, columns = ['label', 'image']) 
        
	def generate(self):
		for category in self.category_dirs:
			category_path = os.path.join(DataGenerator.DATASET_DIR, category)
			if not os.path.isdir(category_path):
				print("no es directorio valido")
				continue
			images = os.listdir(category_path)
			bunch = pd.DataFrame({}, columns = ['label', 'image']) 
			for img in images:
				imgArr = cv2.imread("pokemon\\"+str(category)+"\\"+str(img))
				bunch = bunch.append({'label' : category.upper() , 'image' : imgArr},ignore_index=True)
				#self.data = self.data.append({'label' : category.upper() , 'image' : imgArr},ignore_index=True)
			msk = np.random.rand(len(bunch)) < 0.8
			self.data_train = self.data_train.append(bunch[msk])
			self.data_test = self.data_test.append(bunch[~msk])

In [ ]:
def createFolderIfNotExists(path):
    try:
        os.mkdir(path)
    except FileExistsError:
        #print("Directory data_generated already exists")
        return

In [32]:
DATASET_DIR = "C:\\Users\\ASUS\\Documents\\Proyectos\\pokedex\\pokemon"
numberAnomalies = 0
for category in os.listdir(DATASET_DIR):
    category_path = os.path.join(DataGenerator.DATASET_DIR, category)
    if not os.path.isdir(category_path):
        print("no es directorio valido")
        continue
    images = os.listdir(category_path)
    totalImgs = len(images)
    counter = 0
    path = "data"
    createFolderIfNotExists(path)
    for img in images:
        tempPath = path
        #print(counter / totalImgs)
        if(counter / totalImgs < 0.7):
            tempPath += "\\train"
        elif(counter / totalImgs < 0.85):
            tempPath += "\\test"
        else:
            tempPath += "\\validation"
        createFolderIfNotExists(tempPath)
        tempPath += "\\"+str(category)
        createFolderIfNotExists(tempPath)
        tempPath += "\\"+str(img)
        imgArr = cv2.imread("pokemon\\"+str(category)+"\\"+str(img))
        try:
            cv2.imwrite(tempPath, imgArr) 
        except:
            numberAnomalies += 1
                
        counter += 1
print("numberAnomalies: "+str(numberAnomalies))

numberAnomalies: 33


In [40]:
datagen = keras.preprocessing.image.ImageDataGenerator()

In [41]:
train_it = datagen.flow_from_directory('data\\train\\', class_mode='categorical', batch_size=64)
val_it = datagen.flow_from_directory('data\\validation\\', class_mode='categorical', batch_size=64)
test_it = datagen.flow_from_directory('data\\test\\', class_mode='categorical', batch_size=64)

Found 7529 images belonging to 149 classes.
Found 1524 images belonging to 149 classes.
Found 1604 images belonging to 149 classes.
